# `Machine Learning ZoomCamp`
## `HW1_Chapter 3: Classification`

### Aileah Gotladera
-----------------------------------

In [1]:
import sys
print(sys.executable)
import os
import numpy as np
import pandas as pd

C:\Users\LXPH\anaconda3\envs\MachineLearning\python.exe


In [17]:
# Load the data
file = 'C:/Users/LXPH/OneDrive - Lexcode/Documents/ML ZoomCamp/Data/Homework/ch3_Classification_hw1data.csv'
orig_data = pd.read_csv(file)
print(orig_data.shape)
orig_data.head()

(11914, 16)


,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


### **Data Preparation and Data Cleaning**

In [19]:
# Column name transformation/cleaning
orig_data.columns = orig_data.columns.str.lower().str.replace(' ', '_')
orig_data.columns

Index(['make', 'model', 'year', 'engine_fuel_type', 'engine_hp',
       'engine_cylinders', 'transmission_type', 'driven_wheels',
       'number_of_doors', 'market_category', 'vehicle_size', 'vehicle_style',
       'highway_mpg', 'city_mpg', 'popularity', 'msrp'],
      dtype='object')

In [20]:
# Selecting needed columns
data = orig_data[['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style',
       'highway_mpg', 'city_mpg']]

In [21]:
# Checking dtypes of columns
data.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
dtype: object

In [38]:
# Creating categorical and numerical columns list
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

In [22]:
# Checking null values
data.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
dtype: int64

In [23]:
# Replacing null values with 0
data = data.fillna(0)
data.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
dtype: int64

In [29]:
# Rename MSRP to price (use orig_data df)
orig_data.rename(columns={'msrp':'price'}, inplace=True)
price_col = orig_data['price']
price_col.head()

0    46135
1    40650
2    36350
3    29450
4    34500
Name: price, dtype: int64

## **`Q1`**
* What is the most frequent observation (mode) for the column transmission_type?

In [32]:
q1 = data['transmission_type'].mode()[0]
print(f'Q1 Answer: {q1}')

Q1 Answer: AUTOMATIC


## **`Q2`**
* What are the two features that have the biggest correlation in this dataset?

In [56]:
# Calculating the correlation matrix
correlatioon_matrix = data[numerical].corr()
correlatioon_matrix

# Find 2 features with the highes correlation
max_cor = correlatioon_matrix.abs().unstack()
max_cor = max_cor[max_cor !=1].idxmax()


feature1, feature2 = max_cor

print(f'Q2 Answer: {feature1} and {feature2}')

Q2 Answer: highway_mpg and city_mpg


------------
**Make `price` binary**

In [63]:
# Create the "above_average" column based on the condition
data['above_average'] = (price_col > price_col.mean()).astype(int)

array([1, 0])

**Split the data**

In [161]:
from sklearn.model_selection import train_test_split

# Define features and target 
X = data.drop('above_average', axis = 1)
y = data['above_average']

# Split the data into train, validation and test sets
# Use a 60%/20%/20% distribution with a random seed of 42
X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check shapes
print(f'Train Set shape: {X_train.shape}')
print(f'Valication Set shape: {X_val.shape}')
print(f'Test Set shape: {X_test.shape}')

Train Set shape: (7148, 9)
Valication Set shape: (2383, 9)
Test Set shape: (2383, 9)


## **`Q3`**
* Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.
* Round the scores to 2 decimals using round(score, 2). <br><br>
Which of the variables has the lowest mutual information score?

In [129]:
from sklearn.metrics import mutual_info_score

mi_scores = {}

# Calculating mutual infromation scores per categorical feature
for feature in categorical:
    mi_score = mutual_info_score(X_train[feature], y_train)
    mi_scores[feature] = round(mi_score, 2)

# Sorting the dictionary
sorted_mi_scores = dict(sorted(mi_scores.items(), key=lambda x: x[1], reverse=True))
for feature, score in sorted_mi_scores.items():
    print(f'{feature}:{score}')


model:0.46
make:0.24
vehicle_style:0.08
transmission_type:0.02


In [130]:
# Trying different approach (same with the video)
from sklearn.metrics import mutual_info_score

def mutual_info_ave_score(series):
    mis = mutual_info_score(series, y_train)
    return round(mis,2)

mi_scores = X_train[categorical].apply(mutual_info_ave_score)
mi_scores.sort_values(ascending=False)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [89]:
mi_lowest = list(sorted_mi_scores.keys())[-1]

In [90]:
print(f'Q3 Answer: {mi_lowest}')

Q3 Answer: transmission_type


## **`Q4`**
**Train a Logistic Regression** <br>
* emember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
        * model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [135]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Creating a ColumnTransformer for one-hot encoding
preprocessor = ColumnTransformer(
transformers=[('cat', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), categorical)], remainder='passthrough')

# Creating the logistic Regression Model with specified parameters
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

# Creating a pipeline including data preprocessing and model fitting
pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])

# Combining training and validation data for one-hot encoding
X_combined = pd.concat([X_train, X_val], axis=0)
y_combined = pd.concat([y_train, y_val], axis=0)

# Fit the model on the trai9ning set
pipeline.fit(X_combined, y_combined)


# Predict on the validation dataset
y_val_pred = pipeline.predict(X_val)


# Calculating the accuracy on the validation dataset (rounded to 2 decimal places)
accuracy = round(accuracy_score(y_val, y_val_pred), 2)

print(f'Q4 Answer: {accuracy}')

C:\Users\LXPH\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Q4 Answer: 0.95


In [136]:
X_combined.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg'],
      dtype='object')

## **`Q5`**
Finding the least useful feature using the *feature elimination* technique
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

In [174]:
print('Skipped. Still debugging.')

Skipped. Still debugging.


## **`Q6`**
* We'll need to use the original column price. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
* This model also has a parameter alpha. 
    * Let's try the following values: [0, 0.01, 0.1, 1, 10].
* Round your RMSE scores to 3 decimal digits.

In [170]:
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

data['price'] = np.log1p(orig_data['price'])

# Split the data into training and validation sets
X = data[numerical]
y = data['above_average']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


scores = {}

for alpha  in [0, 0.01, 0.1, 1, 10]:
    ridge = Ridge(alpha=alpha, solver='sag', random_state=42)
    ridge.fit(X_train, y_train)
    y_val_pred = ridge.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val, y_val_pred))
    scores[alpha] = round(rmse, 3)
    
for alpha, rmse in scores.items():
    print(f'{alpha}: {rmse}')

0: 0.323
0.01: 0.323
0.1: 0.323
1: 0.323
10: 0.323


In [171]:
scores.values()

dict_values([0.323, 0.323, 0.323, 0.323, 0.323])

In [173]:
print(f'Q6 Answer: 0')

Q6 Answer: 0
